In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
from qubex.pulse import FlatTop, PulseSchedule
from qubex.simulator import Coupling, MultiControl, Simulator, System, Transmon

In [ ]:
transmons = [
    Transmon(
        label="Q08",
        dimension=3,
        frequency=7.646,
        anharmonicity=-0.356,
        decay_rate=0.00005,
        dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q09",
        dimension=3,
        frequency=8.442,
        anharmonicity=-0.448,
        decay_rate=0.00005,
        dephasing_rate=0.00005,
    ),
]

couplings = [
    Coupling(pair=("Q08", "Q09"), strength=0.01),
]

system = System(
    transmons=transmons,
    couplings=couplings,
)

In [ ]:
system.hamiltonian

In [ ]:
simulator = Simulator(system)

In [ ]:
initial_state = system.state(
    {
        "Q08": "0",
        "Q09": "0",
    },
)

In [ ]:
duration = 30
tau = 10
rx180 = FlatTop(
    duration=duration,
    amplitude=np.pi / (duration - tau),
    tau=tau,
)
ry180 = rx180.shifted(np.pi / 2)

with PulseSchedule(["Q08", "Q09"]) as ps:
    ps.add("Q08", rx180)
    ps.barrier()
    ps.add("Q08", ry180)
    ps.add("Q09", rx180)
    ps.add("Q09", ry180)

ps.plot()

waveforms = ps.get_sampled_sequences()
waveforms

In [ ]:
target = transmons[0]

detuning = 0.001

control = MultiControl(
    frequencies={
        "Q08": transmons[0].frequency + detuning,
        "Q09": transmons[1].frequency + detuning,
    },
    waveforms=waveforms,
    sampling_period=2.0,
    steps_per_sample=10,
)
control.plot()

In [ ]:
result = simulator.simulate(
    control=control,
    initial_state=initial_state,
)

In [ ]:
final_state = result.states[-1]
final_state

In [ ]:
final_state_target = result.substates("Q08")[-1]
final_state_target

In [ ]:
result.plot_population_dynamics()
result.plot_population_dynamics("Q08")
result.plot_population_dynamics("Q09")

In [ ]:
result.show_last_population("Q08")

In [ ]:
result.display_bloch_sphere("Q08")
result.display_bloch_sphere("Q09")